## LOGG 3
### 1. This notebook covers:
Fetching historical ERA5 weather data (temperature, precipitation, wind) using the Open-Meteo API.
Implementing robust SPC (Statistical Process Control) limits using median and MAD (Median Absolute Deviation).
Applying LOF (Local Outlier Factor) to detect anomalies in precipitation data.
Performing STL decomposition (Seasonal-Trend decomposition using Loess) on energy production time series.
Creating spectrograms using SciPy and Plotly to reveal dominant frequency patterns in production data.
Building and restructuring a Streamlit application for interactive visualization and analysis.


### 2. How was the restructuring of Streamlit-project done?

- Rebuilt the Streamlit dashboard to make the project modular, scalable, and connected directly to live APIs instead of static CSV files.
- Restructured the app into multiple pages with separate Python files.
- Created new folders for organization (e.g., pages/, modules/, notebook/).
- Removed outdated Open-Meteo CSV files and replaced them with API calls.
- Added error handling for missing data.
- Extended the dashboard with new pages for SPC, LOF, STL, and spectrogram analysis.
- Updated README.md and requirements.txt to reflect the new project structure.

### 3. Challenges
- Avoiding repeated data loading across pages, while allowing each page to work independently.
    → To solve this, a shared select_area component was created and reused across the entire Streamlit app, with a default value set to "NO1" to ensure functionality even before user interaction.
- Maintaining consistent time zones and data frequencies between ERA5 and production datasets.


### 4. How was AI used? 
Translating and rewriting all code and comments into English.
Explaining advanced concepts and guidance (e.g., STL decomposition, robust statistics, DCT filtering) in simple, applicable terms.
Assisting in documentation updates — ensuring that README.md, requirements.txt, and code comments were consistent and clear.


 ## 1.1 Import Libraries

In [2]:
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import STL
import requests
import plotly.express as px
import numpy as np
from scipy.fft import dct, idct
from sklearn.neighbors import LocalOutlierFactor
from datetime import date
from time import sleep
from scipy import signal

/Users/pialoschbrandt/anaconda3/envs/streamlit/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 1.2 Define a dataframe


In [10]:
# Dataframe som kobler prisområde → by → geografisk posisjon (koordinater).
data = {
    "price_area": ["NO1", "NO2", "NO3", "NO4", "NO5"],
    "city": ["Oslo", "Kristiansand", "Trondheim", "Tromsø", "Bergen"],
    "latitude": [59.9139, 58.1467, 63.4305, 69.6492, 60.3929],
    "longitude": [10.7522, 7.9956, 10.3951, 18.9560, 5.3240]
}

cities_df = pd.DataFrame(data)
print(cities_df)


  price_area          city  latitude  longitude
0        NO1          Oslo   59.9139    10.7522
1        NO2  Kristiansand   58.1467     7.9956
2        NO3     Trondheim   63.4305    10.3951
3        NO4        Tromsø   69.6492    18.9560
4        NO5        Bergen   60.3929     5.3240


## 1.3 Function to fetch weather data from API

In [11]:
# Function to fetch weather data from the Open-Meteo ERA5 API
def fetch_era5_data(latitude, longitude, year):
    """
    Inputs:
        latitude (float): Latitude in decimal degrees
        longitude (float): Longitude in decimal degrees
        year (int): Year of interest
    
    Fetches historical weather data (ERA5 reanalysis) from the Open-Meteo API
    for a given location and year.
    
    Returns:
        pandas.DataFrame: Hourly weather data for the selected year
    """
    start_date = f"{year}-01-01"  # Start date for the selected year
    end_date = f"{year}-12-31"    # End date for the selected year
    
    # Build the URL for the Open-Meteo ERA5 API request
    # Includes coordinates, time range, and hourly weather variables
    # (temperature, precipitation, wind speed, wind gusts, and wind direction)
    # with automatic timezone detection
    url = (
        "https://archive-api.open-meteo.com/v1/era5?"
        f"latitude={latitude}&longitude={longitude}&"
        f"start_date={start_date}&end_date={end_date}&"
        "hourly=temperature_2m,precipitation,wind_speed_10m,wind_gusts_10m,wind_direction_10m&"
        "timezone=auto"
    )

    response = requests.get(url)       # Send GET request to the API
    response.raise_for_status()        # Raise an error if the request failed
    data = response.json()             # Convert the JSON response to a Python dictionary
    
    # Create a pandas DataFrame from the hourly data
    df = pd.DataFrame(data["hourly"])  
    df["time"] = pd.to_datetime(df["time"])  # Convert 'time' column to datetime objects
    
    return df  # Return the DataFrame with weather data


# Example: Fetch data for Bergen (Norway) in 2019
bergen = fetch_era5_data(60.39, 5.32, 2019)
print(bergen.head())  # Display the first few rows


                 time  temperature_2m  precipitation  wind_speed_10m  \
0 2019-01-01 00:00:00             5.7            0.7            37.0   
1 2019-01-01 01:00:00             5.8            0.2            41.0   
2 2019-01-01 02:00:00             6.1            0.7            42.0   
3 2019-01-01 03:00:00             6.3            0.5            40.9   
4 2019-01-01 04:00:00             5.8            1.1            41.2   

   wind_gusts_10m  wind_direction_10m  
0            99.7                 263  
1           107.3                 278  
2           112.0                 286  
3           105.8                 298  
4           110.2                 315  


Hva er kolonnene?

time: tidspunkt for målingen (lokal tid)
temperature_2m: lufttemperatur 2 meter over bakken
precipitation: nedbør per time (regn + snø) (mm/time)
wind_speed_10m: gjennomsnittlig vindhastighet 10 meter over bakken (km/t)
wind_gusts_10m: maksimale vindkast 10 meter over bakken (km/t)
wind_direction_10m: vindretning (grader fra nord, med klokka)


In [17]:
#Plots temperaturs in Bergen 2019
fig = px.line(
    bergen,
    x="time",
    y="temperature_2m",
    title="Temperatur i Bergen (2019)",
    labels={
        "time": "Tid",
        "temperature_2m": "Temperatur (°C)"
    }
)

fig.update_layout(
    template="plotly_white",
    hovermode="x unified",
    title_x=0.5
)

fig.show()


In [22]:
# Function to determine the season based on the month number
def finn_aarstid(mnd):
    if mnd in [12, 1, 2]:
        return "Vinter"   # Winter: December, January, February
    elif mnd in [3, 4, 5]:
        return "Vår"      # Spring: March, April, May
    elif mnd in [6, 7, 8]:
        return "Sommer"   # Summer: June, July, August
    else:
        return "Høst"     # Autumn: September, October, November

# Add month and season columns to the DataFrame
bergen["month"] = bergen["time"].dt.month           # Extract month number from datetime
bergen["season"] = bergen["month"].apply(finn_aarstid)  # Map each month to its season

# Create an interactive line chart using Plotly Express
fig = px.line(
    bergen,
    x="time",                        # X-axis: time (hourly)
    y="temperature_2m",              # Y-axis: temperature in °C
    color="season",                  # Line color by season
    title="Temperatur i Bergen (2019) fordelt på årstider",  # Chart title
    labels={
        "time": "Tid",
        "temperature_2m": "Temperatur (°C)",
        "season": "Årstid"
    }
)

# Apply clean white theme and center the title
fig.update_layout(template="plotly_white", title_x=0.5)

# Display the interactive chart
fig.show()



## 1.4 Robust Statistics for SPC Limits

To find control limits, robust measures of central tendency and dispersion are used:
- Median instead of mean
- Median Absolute Deviation (MAD) instead of standard deviation

Why? 
These are less sensitive to extreme values (outliers) and are therefore better suited for weather data.
The SPC limits are calculated as:

\[
Nedre = median - k_\sigma \cdot 1.4826 \cdot MAD, \quad
Øvre = median + k_\sigma \cdot 1.4826 \cdot MAD
\]


In [35]:
def analyser_temperatur_dct_spc(df,
                                verdi_kolonne="temperature_2m",
                                tids_kolonne="time",
                                cutoff_dager=90,
                                k_sigma=3):
    """
    Performs high-pass filtering of temperature data using DCT
    to create seasonally adjusted temperature variations (SATV),
    and detects outliers based on robust SPC limits (median ± k·1.4826·MAD).

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the columns 'time' and 'temperature_2m'.
    verdi_kolonne : str
        Column name for temperature (default: 'temperature_2m')
    tids_kolonne : str
        Column name for timestamp (default: 'time')
    cutoff_dager : int
        Frequency cutoff for the DCT (number of days considered a "slow trend").
        Default is 90 days.
    k_sigma : float
        Number of standard deviations for SPC limits (default 3).

    Returns
    ----------
    fig : plotly.graph_objects.Figure
        Plot showing temperature with marked outliers.
    oppsummering : dict
        Key metrics: median, sigma, limits, number, and share of outliers.
    """

    # --- 1. Prepare data ---
    s = (df
         .set_index(tids_kolonne)
         .sort_index()[verdi_kolonne]
         .asfreq("1h")
         .interpolate("time"))

    x = s.to_numpy()
    N = len(x)

    # --- 2. Discrete Cosine Transform (DCT) ---
    c = dct(x, type=2, norm="ortho")

    # --- 3. Remove low-frequency components (trend / seasonality) ---
    T_timer = cutoff_dager * 24
    k_cut = int(np.floor(2 * N / T_timer))
    k_cut = max(k_cut, 1)
    c_hp = c.copy()
    c_hp[:k_cut] = 0.0

    # --- 4. Inverse DCT → high-frequency component (SATV) ---
    satv = idct(c_hp, type=2, norm="ortho")

    # --- 5. Robust statistics (median ± k * MAD) ---
    median = np.median(satv)
    mad = np.median(np.abs(satv - median))
    sigma = 1.4826 * mad if mad > 0 else np.std(satv)
    lo, hi = median - k_sigma * sigma, median + k_sigma * sigma

    # --- 6. Identify outliers ---
    is_outlier = (satv < lo) | (satv > hi)

    # --- 7. Plot temperature + outliers ---
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df[tids_kolonne],
        y=df[verdi_kolonne],
        mode="lines",
        name="Temperature (°C)",
        line=dict(color="steelblue")
    ))
    fig.add_trace(go.Scatter(
        x=df[tids_kolonne][is_outlier],
        y=df[verdi_kolonne][is_outlier],
        mode="markers",
        name="Outliers (SATV ±3σ)",
        marker=dict(color="red", size=5)
    ))

    fig.update_layout(
        title="Temperature with SPC Outliers (DCT-based SATV)",
        xaxis_title="Time",
        yaxis_title="Temperature (°C)",
        template="plotly_white",
        hovermode="x unified",
        title_x=0.5
    )

    # --- 8. Summary statistics ---
    oppsummering = {
        "median_SATV": float(median),
        "sigma_SATV": float(sigma),
        "nedre_grense": float(lo),
        "ovre_grense": float(hi),
        "antall_outliers": int(is_outlier.sum()),
        "andel_outliers_%": round(100 * is_outlier.mean(), 2)
    }

    return fig, oppsummering


In [36]:
# Eksempel: analyser temperaturdata for Bergen 2019
fig, oppsummering = analyser_temperatur_dct_spc(bergen,
                                                verdi_kolonne="temperature_2m",
                                                tids_kolonne="time",
                                                cutoff_dager=90,
                                                k_sigma=3)

fig.show()
print(oppsummering)


{'median_SATV': -0.2785385181648098, 'sigma_SATV': 3.232761441779269, 'nedre_grense': -9.976822843502617, 'ovre_grense': 9.419745807172998, 'antall_outliers': 135, 'andel_outliers_%': np.float64(1.54)}


# EXPLANATION OF THE PLOT:

The red-marked points show outliers — that is, moments when the seasonally adjusted temperature variation (SATV) lies outside the robust SPC limits (±3 σ).
This means that the temperature at these times changed unusually quickly compared to what is normal for the season.
In other words, the red points indicate extreme or atypical warm or cold shocks that clearly deviate from the usual seasonal pattern.

# EXPLANATION OF VARIABLES:

- median_SATV shows the median of the seasonally adjusted temperature variations (SATV), meaning that the values on average are centered around –0.28 °C — indicating no clear bias relative to the seasonal trend.
- sigma_SATV shows the robust standard deviation (1.4826 × MAD), meaning that typical rapid temperature variations are about ±3.23 °C.
- nedre_grense (lower limit) shows median – 3·σ, meaning that SATV values lower than –9.98 °C are considered unusually cold deviations.
- ovre_grense (upper limit) shows median + 3·σ, meaning that SATV values higher than +9.42 °C are considered unusually rapid warm increases.
- antall_outliers shows the number of time points where SATV is outside the SPC limits, meaning that 135 hours in 2019 experienced extreme temperature changes.
- andel_outliers_% shows the proportion of all observations that are outliers, meaning that about 1.54 % of the year’s hours had unusually rapid temperature variations relative to the seasonal trend.

## 1.6 LOF - Analysis

In [26]:
def analyze_precipitation_lof(df,
                              value_column="precipitation",
                              time_column="time",
                              outlier_fraction=0.01):
    """
    Analyzes precipitation data using the Local Outlier Factor (LOF)
    to identify unusual precipitation events (anomalies).

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns 'time' and 'precipitation'.
    value_column : str
        Column name for precipitation (default: 'precipitation')
    time_column : str
        Column name for timestamp (default: 'time')
    outlier_fraction : float
        Expected fraction of outliers (default 0.01 = 1%)

    Returns
    ----------
    fig : plotly.graph_objects.Figure
        Plot showing precipitation and marked anomalies.
    summary : dict
        Key metrics: number and share of outliers, LOF statistics.
    """

    # --- 1. Prepare data ---
    # Set time as index, ensure hourly frequency, interpolate missing values
    s = (df
         .set_index(time_column)
         .sort_index()[value_column]
         .asfreq("1h")
         .interpolate("time")
         .fillna(0.0))

    x = s.to_numpy().reshape(-1, 1)  # Convert to NumPy array for LOF input

    # --- 2. Local Outlier Factor analysis ---
    lof = LocalOutlierFactor(
        contamination=outlier_fraction,  # expected fraction of outliers
        n_neighbors=50                   # number of neighbors for local density estimation
    )

    labels = lof.fit_predict(x)              # Fit model and predict normal/outlier labels
    scores = -lof.negative_outlier_factor_   # Higher score = more anomalous

    # --- 3. Identify outliers ---
    is_outlier = labels == -1

    # --- 4. Plot results ---
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df[time_column],
        y=df[value_column],
        mode="lines",
        name="Precipitation (mm/h)",
        line=dict(color="royalblue")
    ))
    fig.add_trace(go.Scatter(
        x=df[time_column][is_outlier],
        y=df[value_column][is_outlier],
        mode="markers",
        name="Anomalies (LOF)",
        marker=dict(color="red", size=6)
    ))

    fig.update_layout(
        title=f"Precipitation with LOF anomalies (fraction={outlier_fraction*100:.1f}%)",
        xaxis_title="Time",
        yaxis_title="Precipitation (mm/h)",
        template="plotly_white",
        hovermode="x unified",
        title_x=0.5
    )

    # --- 5. Summary statistics ---
    summary = {
        "total_observations": int(len(x)),
        "number_of_outliers": int(is_outlier.sum()),
        "outlier_fraction_%": round(100 * is_outlier.mean(), 2),
        "mean_LOF_score": round(scores.mean(), 3),
        "max_LOF_score": round(scores.max(), 3)
    }

    return fig, summary


In [28]:
# Run LOF analysis on Bergen precipitation data
fig_lof, summary_lof = analyze_precipitation_lof(
    bergen,
    value_column="precipitation",  # column with precipitation data
    time_column="time",            # timestamp column
    outlier_fraction=0.01          # expected share of outliers (1%)
)

# Show interactive Plotly figure
fig_lof.show()

# Print summary statistics
print(summary_lof)


{'total_observations': 8760, 'number_of_outliers': 86, 'outlier_fraction_%': np.float64(0.98), 'mean_LOF_score': np.float64(1.005), 'max_LOF_score': np.float64(8.672)}


FORKLARING:

- LOF (Local Outlier Factor) ser på hvor tett en observasjon ligger i forhold til naboene sine.
Hvis en måling er “langt unna” naboene (f.eks. ekstrem nedbør eller null når det burde regne), får den høy LOF-score.

- Parametret andel_outliers bestemmer hvor streng metoden er:
0.01 = forvent ca. 1 % anomalier. kan justere til f.eks. 0.02 for å gjøre den mer følsom.

- Outliers vises som røde prikker på tidsserien.

## 1.7 STL - Seasonal-Trend decomposition using Loess

In [ ]:

def analyser_stl_produksjon(df,
                            verdi_kolonne="production",
                            tids_kolonne="time",
                            prisområde="NO5",
                            produksjonsgruppe="total",
                            periode_lengde=24*7,  # én uke (timesdata)
                            seasonal_smoother=13,
                            trend_smoother=101,
                            robust=True):
    """
    Utfører STL-dekomponering (Seasonal-Trend using LOESS) på produksjonsdata.

    Parametre
    ----------
    df : pd.DataFrame
        DataFrame med kolonnene 'time', 'production', 'price_area', 'production_group'.
    verdi_kolonne : str
        Kolonnenavn for produksjonsdata (standard: 'production').
    tids_kolonne : str
        Kolonnenavn for tid (standard: 'time').
    prisområde : str
        Elektrisitetsprisområde, f.eks. 'NO1'–'NO5'.
    produksjonsgruppe : str
        Produksjonstype (f.eks. 'total', 'hydro', 'wind', etc.)
    periode_lengde : int
        Sesongperiode (f.eks. 168 timer = 1 uke).
    seasonal_smoother : int
        Lengde på sesongutjevner (s).
    trend_smoother : int
        Lengde på trendutjevner (t).
    robust : bool
        Bruk robust LOESS (reduser påvirkning av outliers).

    Returnerer
    ----------
    fig : plotly.graph_objects.Figure
        Figur med STL-dekomponering.
    """

    # --- Filtrer data for valgt prisområde og produksjonsgruppe ---
    df_filtered = df[(df["price_area"] == prisområde) &
                     (df["production_group"] == produksjonsgruppe)].copy()

    # --- Forbered tidsserie ---
    s = (df_filtered
         .set_index(tids_kolonne)
         .sort_index()[verdi_kolonne]
         .asfreq("1h")
         .interpolate("time"))

    # --- Utfør STL-dekomponering ---
    stl = STL(s,
              period=periode_lengde,
              seasonal=seasonal_smoother,
              trend=trend_smoother,
              robust=robust)
    res = stl.fit()

    # --- Lag plot ---
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=s.index, y=s, name="Original", line=dict(color="blue")))
    fig.add_trace(go.Scatter(x=s.index, y=res.trend, name="Trend", line=dict(color="orange")))
    fig.add_trace(go.Scatter(x=s.index, y=res.seasonal, name="Sesong", line=dict(color="green")))
    fig.add_trace(go.Scatter(x=s.index, y=res.resid, name="Residual", line=dict(color="gray")))

    fig.update_layout(
        title=f"STL-dekomponering – {prisområde}, {produksjonsgruppe}",
        xaxis_title="Tid",
        yaxis_title="Produksjon (MW)",
        template="plotly_white",
        hovermode="x unified",
        title_x=0.5
    )

    return fig


In [29]:
base_url = "https://api.elhub.no/energy-data/v0/price-areas"
params = {'dataset': 'PRODUCTION_PER_GROUP_MBA_HOUR'}
all_data = []

for month in range(1, 13):
    start = date(2021, month, 1)
    end = date(2022, 1, 1) if month == 12 else date(2021, month + 1, 1)

    # add timezone information
    params['startDate'] = f"{start.isoformat()}T00:00:00+02:00"
    params['endDate']   = f"{end.isoformat()}T00:00:00+02:00"

    print(f"Fetching {start.strftime('%B %Y')} ...", end=" ")

    r = requests.get(base_url, params=params)
    r.raise_for_status()
    data = r.json().get('data', [])

    rows = []
    for d in data:
        attr = d['attributes']
        for p in attr['productionPerGroupMbaHour']:
            rows.append({
                'country': attr.get('country'),
                'priceArea': p.get('priceArea'),
                'productionGroup': p.get('productionGroup'),
                'quantityKwh': p.get('quantityKwh'),
                'startTime': p.get('startTime'),
                'endTime': p.get('endTime'),
                'lastUpdatedTime': p.get('lastUpdatedTime')
            })

    if rows:
        df = pd.DataFrame(rows)
        all_data.append(df)
        print(f"✅ {len(df)} rows")
    else:
        print("⚠️ no data")

# combine all months into one DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# convert timestamp columns
for col in ['startTime', 'endTime', 'lastUpdatedTime']:
    df_all[col] = pd.to_datetime(df_all[col], errors='coerce')

print("\n✅ Done!")
print(f"Total retrieved: {len(df_all)} rows")
print(f"Time period: {df_all['startTime'].min()} → {df_all['startTime'].max()}")
print("Price areas:", df_all['priceArea'].unique())
print("Production groups:", df_all['productionGroup'].unique())



Fetching January 2021 ... ✅ 17856 rows
Fetching February 2021 ... ✅ 16128 rows
Fetching March 2021 ... ✅ 17832 rows
Fetching April 2021 ... ✅ 17280 rows
Fetching May 2021 ... ✅ 17856 rows
Fetching June 2021 ... ✅ 17976 rows
Fetching July 2021 ... ✅ 18600 rows
Fetching August 2021 ... ✅ 18600 rows
Fetching September 2021 ... ✅ 18000 rows
Fetching October 2021 ... ✅ 18625 rows
Fetching November 2021 ... ✅ 18000 rows
Fetching December 2021 ... ✅ 18600 rows


/var/folders/wq/bswb0jn17m3gt1jh6tflwvhc0000gn/T/ipykernel_55612/3257218840.py:45: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

/var/folders/wq/bswb0jn17m3gt1jh6tflwvhc0000gn/T/ipykernel_55612/3257218840.py:45: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

/var/folders/wq/bswb0jn17m3gt1jh6tflwvhc0000gn/T/ipykernel_55612/3257218840.py:45: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error 


✅ Done!
Total retrieved: 215353 rows
Time period: 2021-01-01 00:00:00+01:00 → 2021-12-31 23:00:00+01:00
Price areas: ['NO1' 'NO2' 'NO3' 'NO4' 'NO5']
Production groups: ['hydro' 'other' 'solar' 'thermal' 'wind']


In [30]:
def stl_decomposition_plot(
    df,
    price_area="NO1",
    production_group="hydro",
    period=24,
    seasonal=13,
    trend=31,
    robust=True
):
    # --- 1. Check that all required columns exist ---
    required_cols = {"priceArea", "productionGroup", "quantityKwh", "startTime"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"Missing columns: {required_cols - set(df.columns)}")

    # --- 2. Filter data for the selected price area and production group ---
    df_filtered = df[
        (df["priceArea"].str.upper() == price_area.upper()) &
        (df["productionGroup"].str.lower() == production_group.lower())
    ].copy()

    if df_filtered.empty:
        raise ValueError(f"No data found for {price_area} / {production_group}")

    # --- 3. Convert timestamps to datetime and remove timezone ---
    df_filtered["startTime"] = (
        pd.to_datetime(df_filtered["startTime"], utc=True, errors="coerce")
        .dt.tz_convert(None)
    )

    # --- 4. Sort and set time index ---
    df_filtered = df_filtered.sort_values("startTime")
    df_filtered = df_filtered.set_index("startTime")

    # --- 5. Resample hourly and interpolate missing values ---
    ts = df_filtered["quantityKwh"].resample("h").mean().interpolate()

    # --- 6. Perform STL decomposition ---
    from statsmodels.tsa.seasonal import STL
    import plotly.graph_objects as go

    # STL = Seasonal-Trend decomposition using Loess
    stl = STL(ts, period=period, seasonal=seasonal, trend=trend, robust=robust)
    result = stl.fit()

    # --- 7. Create interactive Plotly figure ---
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ts.index, y=ts, name="Original", line=dict(color="blue")))
    fig.add_trace(go.Scatter(x=ts.index, y=result.trend, name="Trend", line=dict(color="red")))
    fig.add_trace(go.Scatter(x=ts.index, y=result.seasonal, name="Seasonal", line=dict(color="orange")))
    fig.add_trace(go.Scatter(x=ts.index, y=result.resid, name="Residual", line=dict(color="green")))

    # --- 8. Format layout ---
    fig.update_layout(
        title=f"STL Decomposition for {production_group} in {price_area}",
        xaxis_title="Time",
        yaxis_title="Production (kWh)",
        template="plotly_white",
        hovermode="x unified"
    )

    return fig  # Return the interactive figure



In [31]:
fig = stl_decomposition_plot(df_all, price_area="NO1", production_group="hydro")
fig.show()



- Blå (Original): Viser den faktiske timesbaserte vannkraftproduksjonen gjennom hele 2021, med tydelige døgn- og sesongvariasjoner.
- Rød (Trend): Viser den langsiktige utviklingen i produksjonen over året, jevnet ut fra daglige svingninger.
- Oransje (Seasonal): Viser det gjentakende døgnmønsteret i produksjonen som følge av regelmessige variasjoner i etterspørsel og drift.
- Grønn (Residual): Viser tilfeldige avvik og uregelmessigheter som ikke fanges opp av trend eller sesong.

## 1.8 Spectrogram

In [32]:
def spectrogram_plot(
    df: pd.DataFrame,
    price_area: str = "NO1",
    production_group: str = "hydro",
    window_length: int = 256,   # number of hours per analysis window
    overlap: int = 128          # number of overlapping hours
) -> go.Figure:
    """
    Create an interactive spectrogram (Plotly) for Elhub production data.

    Expected columns in df: ['priceArea', 'productionGroup', 'quantityKwh', 'startTime'].
    Window length and overlap are specified in hours (the dataset is assumed hourly).
    """

    # --- 1. Validate input ---
    req = {"priceArea", "productionGroup", "quantityKwh", "startTime"}
    if not req.issubset(df.columns):
        raise ValueError(f"Missing columns: {req - set(df.columns)}")

    # --- 2. Filter and prepare the time series (case-insensitive match) ---
    dff = df[
        (df["priceArea"].str.upper() == price_area.upper()) &
        (df["productionGroup"].str.lower() == production_group.lower())
    ].copy()
    if dff.empty:
        raise ValueError(f"No data for {price_area}/{production_group}")

    # Convert timestamps to datetime and remove timezone information
    dff["startTime"] = pd.to_datetime(dff["startTime"], utc=True, errors="coerce").dt.tz_convert(None)
    dff = dff.dropna(subset=["startTime"]).sort_values("startTime").set_index("startTime")

    # --- 3. Resample to hourly frequency and fill any gaps ---
    ts = dff["quantityKwh"].resample("h").mean().interpolate()
    if len(ts) < max(32, window_length):
        raise ValueError(f"Series too short ({len(ts)} points) for chosen window_length={window_length}")

    # --- 4. Ensure valid spectrogram parameters ---
    overlap = min(overlap, window_length - 1)
    fs = 1.0  # sampling rate: 1 measurement per hour

    # --- 5. Compute spectrogram using SciPy ---
    f, t_rel, Sxx = signal.spectrogram(
        ts.values,
        fs=fs,
        nperseg=window_length,
        noverlap=overlap,
        detrend="linear",
        scaling="spectrum",
        window="hann"
    )

    # Convert to decibel (dB) scale for readability
    Sxx_db = 10 * np.log10(Sxx + 1e-12)

    # --- 6. Convert relative time axis (hours from start) to absolute datetimes ---
    t0 = ts.index[0]
    t_abs = t0 + pd.to_timedelta(t_rel, unit="h")

    # --- 7. Convert frequency axis to cycles per day (easier interpretation) ---
    f_per_day = f * 24.0

    # --- 8. Create interactive Plotly heatmap ---
    fig = go.Figure(
        data=go.Heatmap(
            z=Sxx_db,
            x=t_abs,
            y=f_per_day,
            colorscale="Viridis",
            colorbar=dict(title="Power [dB]")
        )
    )

    # --- 9. Format layout and labels ---
    fig.update_layout(
        title=f"Spectrogram for {production_group} in {price_area}",
        xaxis_title="Time",
        yaxis_title="Frequency [cycles/day]",
        template="plotly_white"
    )

    return fig  # return interactive Plotly figure



In [83]:
fig = spectrogram_plot(df_all, price_area="NO1", production_group="hydro",
                       window_length=256, overlap=128)
fig.show()


Tolkning:
- X-akse: Tid gjennom året 2021
- Y-akse: Frekvens i antall sykluser per døgn (hvor ofte produksjonen varierer)
- Farge: Viser styrken på variasjonen (energi/power)
    Gul: Sterk frekvens – tydelige, regelmessige svingninger i produksjonen (dag- og nattvariasjoner)
    Blå/grønn: Svak frekvens – stabile perioder med liten variasjon